In [1]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from specbuffer import SpecReplayBuffer
from buffer import ReplayBuffer
from PPOee import PPO
import gym
from CustomEnv import CustomEnv
from Student_Agent import StudentAgent
from TeacherAgent import TeacherAgent
from ExpertTrajectory import ExpertTrajectory

import contextlib
import network_sim
from stable_baselines3 import PPO as sb3ppo
import pickle
import importlib

2024-04-07 01:58:45.389633: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12


cuda:0


In [16]:
m=importlib.import_module('specbuffer','SpecReplayBuffer')
importlib.reload(m)

<module 'specbuffer' from '/home/tools/DR/MyProject/RILE/specbuffer.py'>

In [2]:
env=CustomEnv('PccNs-v0',1200)

History length: 10
Features: ['sent latency inflation', 'latency ratio', 'send ratio']
Getting min obs for ['sent latency inflation', 'latency ratio', 'send ratio']


In [3]:
et=ExpertTrajectory(1600)

In [4]:
et.load_expert_model('verygood.zip')
et.generate_trajectory(env,40)

400it [00:00, 481.51it/s]                     


In [5]:
class Student(StudentAgent):
    def __init__(self, state_dim, action_dim, env):
        super().__init__(state_dim, action_dim, env)
        
    def generate_trajectory(self, step: int):
        pb=tqdm(range(step))
        for _ in pb:
            s=self.env.reset()
            d=False
            while not d:
                a,action,l,v=self.model.select_action(s)
                s_,r,d,_=self.env.step(a)
                self.model.buffer.store(s[-3:],a,l,s_,r,v,d)
                s=s_
            pb.update()

In [6]:
sa=Student(30,1,env)

In [62]:
sa.replay_buffer.clean()

In [63]:
sa.generate_trajectory(10)

100%|██████████| 10/10 [00:03<00:00,  2.87it/s]


In [42]:
sa.train(10,8)

  0%|          | 0/10 [00:00<?, ?it/s]/home/data/envs/IL/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 10/10 [00:01<00:00,  7.97it/s]


In [79]:
class Teacher(TeacherAgent):
    def __init__(self, state_dim, action_dim, env:CustomEnv, expert_trajectory):
        super().__init__(state_dim, action_dim, env, expert_trajectory)
        self.model = PPO(10, action_dim,
                         256, self.replay_buffer)
        
    def ComputeReward(self):
        pb=tqdm(range(min(self.trajectory_buffer.index,self.trajectory_buffer.buffer_size)))
        for i in pb:
            length=len(self.trajectory_buffer.state[0])
            s=torch.FloatTensor(self.trajectory_buffer.get_state_padding(self.trajectory_buffer.state[int(i/length)],i%length,10,True)[1]).view(-1)
            r=torch.FloatTensor(self.trajectory_buffer.get_reward_padding(self.trajectory_buffer.reward,i,10,length,True)[1]).view(-1)
            reward,tensor_reward,l,v=self.model.select_action(r)
            sa_pair=torch.cat((s.cuda(),tensor_reward.view(-1)),-1)
            self.replay_buffer.store(
                r,
                reward,
                l,
                self.trajectory_buffer.next_state[i],
                self.discriminator.model(sa_pair).detach().cpu().numpy(),
                v,
                self.trajectory_buffer.done[i]
            )
            self.trajectory_buffer.reward[i]=reward
            pb.update()
        self.discriminator.collect_expert()

In [82]:
ta=Teacher(30,1,env,et)

In [83]:
ta.ComputeReward()
#BUG 这里有问题

100%|██████████| 1200/1200 [00:00<00:00, 1264.59it/s]


In [84]:
sa.train(10,8)

100%|██████████| 10/10 [00:01<00:00,  7.82it/s]


In [13]:
ta.model.buffer

In [85]:
ta.train(10,10,10)

  0%|          | 0/10 [00:00<?, ?it/s]

Computing reward...


100%|██████████| 1200/1200 [00:00<00:00, 1210.70it/s]
/home/data/envs/IL/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([1024, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Training Discriminator...
Training PPO...


 10%|█         | 1/10 [00:02<00:24,  2.76s/it]

...

13it [00:24,  2.02s/it]

Computing reward...


100%|██████████| 1200/1200 [00:00<00:00, 1262.37it/s]


Training Discriminator...
Training PPO...


100%|██████████| 10/10 [00:27<00:00,  2.74s/it]


In [15]:
import importlib
module=importlib.import_module('Discriminator','Discriminator.py')


In [16]:
d=module.Discriminator(31,1,8,sa.replay_buffer,et)

In [17]:
d.collect_expert()

In [18]:
d.collect_dist(8)

In [19]:
d.update(100,False)